In [7]:
import pandas as pd
from pandas.io import sql
from scipy.spatial.distance import cosine
from numpy import array
from numpy import percentile
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.switch_backend('agg')
from itertools import combinations
from itertools import permutations

from kmodes import kprototypes

from sklearn.cluster import KMeans
from scipy.stats import itemfreq
import pickle
import warnings
warnings.filterwarnings("ignore")

In [11]:
num_clusters = 10
kproto = kprototypes.KPrototypes(n_clusters=num_clusters, init='Cao', verbose=2)

engine_objects = {}


In [12]:

def loadData():
  customer_data = pd.read_excel('Utilities Product Catalog & Machine Learning Data.xlsx',sheet_name='Utilities ML Data')
  #product_catalog = pd.read_excel('Utilities Product Catalog & Machine Learning Data.xlsx',sheet_name='Utilities Product Catalog')
  return customer_data


def getRecommendations(customer_data):
    customer_data['id'] = customer_data.index

    tempcluster_data = pd.DataFrame()
    tempcluster_data["usage"] = customer_data["Total Usage in kwh"]
    tempcluster_data["usage"].fillna(0, inplace=True)
    tempcluster_data["invoice"] = customer_data["Average Invoice/Month (Static)"]
    tempcluster_data["no_members"] = customer_data["Number of members in family"]
    tempcluster_data["electric_car"] = customer_data["Electric Car (Y/N)"]
    tempcluster_data["locality"] = customer_data["Locality (Zip)"]
    tempcluster_data["customer_since"] = customer_data["Customer Since"]
    tempcluster_data["segment"] = customer_data["Customer segment"]

    customer_since_min = customer_data["Customer Since"].min(0)
    engine_objects['customer_since_min'] = customer_since_min
    customer_since_max = customer_data["Customer Since"].max(0)
    engine_objects['customer_since_max'] =customer_since_max
    no_members_min = customer_data["Number of members in family"].min(0)
    engine_objects['no_members_min'] =no_members_min
    no_members_max = customer_data["Number of members in family"].max(0)
    engine_objects['no_members_max'] =no_members_max
    invoice_min = customer_data["Average Invoice/Month (Static)"].min(0)
    engine_objects['invoice_min'] =invoice_min
    invoice_max = customer_data["Average Invoice/Month (Static)"].max(0)
    engine_objects['invoice_max'] =invoice_max
    usage_min = customer_data["Total Usage in kwh"].min(0)
    engine_objects['usage_min'] =usage_min
    usage_max = customer_data["Total Usage in kwh"].max(0)
    engine_objects['usage_max'] =usage_max

    tempcluster_data["customer_since"] = ((tempcluster_data["customer_since"] - customer_since_min) / (customer_since_max - customer_since_min))
    tempcluster_data["no_members"] = ((tempcluster_data["no_members"] - no_members_min) / (no_members_max - no_members_min))
    tempcluster_data["invoice"] = ((tempcluster_data["invoice"] - invoice_min) / (invoice_max - invoice_min))
    tempcluster_data["usage"] = ((tempcluster_data["usage"] - usage_min) / (usage_max - usage_min))
    tempcluster_data["locality"] = tempcluster_data["locality"].astype(str)

    clusters = kproto.fit_predict(tempcluster_data.as_matrix(), categorical=[3, 4, 6])
    engine_objects['kproto'] = kproto
    clustermap = pd.DataFrame(columns=["id", "cluster"])
    clustermap["id"] = customer_data["id"]
    clustermap["cluster"] = clusters

    b = pd.DataFrame(customer_data['Concatenated Products'].str.split(',').tolist(), index=customer_data['id']).stack()
    b = b.reset_index()[[0, 'id']]  # var1 variable is currently labeled 0
    b.columns = ['Concatenated Products', 'id']  # renaming var1

    del customer_data['Concatenated Products']
    customer_data = pd.merge(customer_data, b, on="id")
    customer_data['Concatenated Products'] = customer_data['Concatenated Products'].str.strip()

    subscription_data = pd.DataFrame()
    subscription_data['id'] = customer_data['id']
    subscription_data['Product'] = customer_data['Concatenated Products']
    subscription_data['supplement'] = 1

    customer_subscriptions_pivot_table = pd.pivot_table(subscription_data, index=["id"], columns=['Product'],
                                                        values="supplement", aggfunc=max, fill_value=0)

    customer_subs_orig = pd.DataFrame(customer_subscriptions_pivot_table.get_values())
    customer_subs_orig.columns = customer_subscriptions_pivot_table.columns.tolist()

    customer_subs = pd.concat([clustermap, customer_subs_orig], axis=1)

    recommendationsmatrix = {}
    recommendationsmatrixnumbers = {}

    for a in range(num_clusters):
        temp_cluster_data = customer_subs[customer_subs["cluster"] == a]
        print(str(a) + "<=================================================")
        del temp_cluster_data["cluster"]
        del temp_cluster_data["id"]
        columns_plans = temp_cluster_data.columns.tolist()
        customer_products = pd.DataFrame(index=columns_plans, columns=columns_plans)
        for i in range(0, len(customer_products.columns)):
            for j in range(0, len(customer_products.columns)):
                customer_products.ix[i, j] = 1 - cosine(temp_cluster_data.ix[:, i], temp_cluster_data.ix[:, j])
        if (len(customer_products.columns) > 8):
            plans_length = 8
        else:
            plans_length = len(customer_products.columns)
        plans_similar = pd.DataFrame(index=customer_products.columns, columns=range(1, (plans_length + 1)))
        plans_similar_number = pd.DataFrame(index=customer_products.columns, columns=range(1, (plans_length + 1)))
        for i in range(0, len(customer_products.columns)):
            plans_similar.ix[i, :plans_length] = customer_products.ix[0:, i].sort_values(ascending=False)[
                                                 :plans_length].index
            plans_similar_number.ix[i, :plans_length] = customer_products.ix[0:, i].sort_values(ascending=False)[
                                                        :plans_length].values
        recommendationsmatrix[a] = plans_similar
        recommendationsmatrixnumbers[a] = plans_similar_number

    recommendationresults = []
    recommendationresultswithpercentage = []

    for a in range(num_clusters):
        for b in range(len(recommendationsmatrix[a])):
            for c in range(1, 9):
                recommendationresultswithpercentage.append(
                    {"cluster_id": a, "match_percent": recommendationsmatrixnumbers[a][c][b],
                     "plan_id": recommendationsmatrix[a].index[b], "recommendation_sequence": c,
                     "recommendation_plan_id": recommendationsmatrix[a][c][b]})

    recommendationresultswithpercentage = pd.DataFrame(recommendationresultswithpercentage)
    engine_objects['recommendationresultswithpercentage'] = recommendationresultswithpercentage

    filehandler_engine_objects = open('engine_objects.obj', 'wb')
    pickle.dump(engine_objects, filehandler_engine_objects)



In [14]:
recommendation_cluster_with_percentage = pd.DataFrame()
tempdata=loadData()
recommendation_cluster_with_percentage = getRecommendations(tempdata)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 2522, ncost: 868.4502846291687
Run: 1, iteration: 2/100, moves: 1010, ncost: 816.6474620763854
Run: 1, iteration: 3/100, moves: 383, ncost: 808.8402668259586
Run: 1, iteration: 4/100, moves: 140, ncost: 807.6831006301529
Run: 1, iteration: 5/100, moves: 37, ncost: 807.5816609712264
Run: 1, iteration: 6/100, moves: 26, ncost: 807.5305294949168
Run: 1, iteration: 7/100, moves: 6, ncost: 807.5257238628202
Run: 1, iteration: 8/100, moves: 8, ncost: 807.5157070744182
Run: 1, iteration: 9/100, moves: 9, ncost: 807.5082811034345
Run: 1, iteration: 10/100, moves: 12, ncost: 807.4902074774395
Run: 1, iteration: 11/100, moves: 14, ncost: 807.4431055322882
Run: 1, iteration: 12/100, moves: 27, ncost: 807.3543366942021
Run: 1, iteration: 13/100, moves: 39, ncost: 807.0906503951999
Run: 1, iteration: 14/100, moves: 43, ncost: 806.8957258704764
Run: 1, iteration: 15/100, moves: 34, ncost:

8<=================================================
9<=================================================
